In [1]:
#basic
import numpy as np 
import pandas as pd
from datetime import timedelta
#viz
import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

#others
import subprocess
from subprocess import check_output
import gc

%matplotlib inline

In [2]:
# train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
stores = pd.read_csv("input/stores.csv")
items = pd.read_csv("input/items.csv")
trans = pd.read_csv("input/transactions.csv")
oil = pd.read_csv("input/oil.csv")
holiday = pd.read_csv("input/holidays_events.csv")
print("done")

done


C:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#check memory use for the two biggest files - train and test
#mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
#print("train dataset uses ",mem_train/ 1024**2," MB")
print("test dataset uses ",mem_test/ 1024**2," MB")
# checking contents in train
test.head()

test dataset uses  27.816246032714844  MB


,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [4]:
# optimize test.csv
# First check the contents of train.csv
print(test.max())
print(test.min())
#check datatypes
print(test.dtypes)

id              126226222
date           2017-08-19
store_nbr              54
item_nbr          2134244
onpromotion          True
dtype: object
id              125497040
date           2017-08-16
store_nbr               1
item_nbr            20289
onpromotion         False
dtype: object
id              int64
date           object
store_nbr       int64
item_nbr        int64
onpromotion    object
dtype: object


In [5]:
#There are only 54 stores
test['store_nbr'] = test['store_nbr'].astype(np.uint8)
# The ID column is a continuous number from 1 to 128867502 in train and 128867503 to 125497040 in test
test['id'] = test['id'].astype(np.uint32)
# item number is unsigned 
test['item_nbr'] = test['item_nbr'].astype(np.uint32)
#Converting the date column to date format
test['date']=pd.to_datetime(test['date'],format="%Y-%m-%d")
#check memory
print(test.memory_usage(index=True))
new_mem_test=test.memory_usage(index=True).sum()
print("test dataset uses ",new_mem_test/ 1024**2," MB after changes")
print("memory saved =",(mem_test-new_mem_test)/ 1024**2," MB")

Index              128
id             2916732
date           5833464
store_nbr       729183
item_nbr       2916732
onpromotion    5833464
dtype: int64
test dataset uses  17.385199546813965  MB after changes
memory saved = 10.431046485900879  MB


In [6]:
# scaling part 1 to the entire dataset
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }

In [7]:
# now scaling it to the entire dataset of train

train_part2=pd.read_csv("input/train.csv",dtype=dtype_dict,usecols=[1,5],parse_dates=['date'], skiprows=range(1, 101688779))
train_part2['Year'] = pd.DatetimeIndex(train_part2['date']).year
train_part2['Month'] = pd.DatetimeIndex(train_part2['date']).month
train_part2['Day'] =pd.DatetimeIndex(train_part2['date']).day.astype(np.uint8)
del(train_part2['date'])
train_part2['Day']=train_part2['Day'].astype(np.uint8)
train_part2['Month']=train_part2['Month'].astype(np.uint8)
train_part2['Year']=train_part2['Year'].astype(np.uint16)

#impute the missing values to be -1
train_part2["onpromotion"].fillna(0, inplace=True)
train_part2["onpromotion"]=train_part2["onpromotion"].astype(np.int8)
print(train_part2.head())
print(train_part2.dtypes)

   onpromotion  Year  Month  Day
0            0  2016     12   31
1            0  2017      1    1
2            0  2017      1    1
3            0  2017      1    1
4            0  2017      1    1
onpromotion      int8
Year           uint16
Month           uint8
Day             uint8
dtype: object


In [8]:
# dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

In [9]:
train_part1 = pd.read_csv("input/train.csv",dtype=dtype_dict, usecols=[0,1,2,3,4], parse_dates=['date'], skiprows=range(1, 101688779))
print(train_part1.dtypes)

id                    uint32
date          datetime64[ns]
store_nbr              uint8
item_nbr              uint32
unit_sales           float32
dtype: object


In [10]:
# joining part one and two
# For people familiar with R , the equivalent of cbind in pandas is the following command
train = pd.concat([train_part1.reset_index(drop=True), train_part2], axis=1)
#drop temp files
del(train_part1)
del(train_part2)
#Further Id is just an indicator column, hence not required for analysis
id=train['id']
del(train['id'])
# check memory
print(train.memory_usage())
#The extracted train.csv file is approx 5 GB
mem_train=5*1024**3
new_mem_train=train.memory_usage().sum()
print("Train dataset uses ",new_mem_train/ 1024**2," MB after changes")
print("memory saved is approx",(mem_train-new_mem_train)/ 1024**2," MB")

Index                128
date           190466096
store_nbr       23808262
item_nbr        95233048
unit_sales      95233048
onpromotion     23808262
Year            47616524
Month           23808262
Day             23808262
dtype: int64
Train dataset uses  499.51733779907227  MB after changes
memory saved is approx 4620.482662200928  MB


In [11]:
import time

dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}
start = time.time()
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion


print("Step 1 started")
end = time.time()
print(f"Step 1 Time:{end - start}")
start = time.time()
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)

del u_dates, u_stores, u_items

print("Step 2 started")
end = time.time()
print(f"Step 2 Time:{end - start}")
start = time.time()
train.loc[:, "unit_sales"].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

print("Step 3 started")
end = time.time()
print(f"Step 3 Time:{end - start}")
#Load test
# test = pd.read_csv("../input/test.csv", usecols=[0,2,3], dtype=dtypes)
test = test.set_index(['item_nbr', 'store_nbr'])
ltest = test.shape[0]
print("Done")

<ipython-input-11-c8ca3e695ed9>:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion


Step 1 started
Step 1 Time:0.40920114517211914
Step 2 started
Step 2 Time:855.025548696518
Step 3 started
Step 3 Time:19.415262460708618
Done


In [12]:
train.to_csv("train_cleaned.csv")

In [13]:
test.to_csv("test_cleaned.csv")

In [ ]:
from __future__ import annotations
import os
import logging

from vectice import Vectice, Experiment
from vectice.__version__ import __version__, __vectice_version__

os.environ["PROJECT_TOKEN"] = "g4Kn72y0t5LMRqr1rqO6"
os.environ["VECTICE_API_ENDPOINT"] = "app.vectice.com"
os.environ[
    "VECTICE_API_TOKEN"
] = "Q31E39q8D.9wbZ09gmjONeE4nPaBldoQ31E39q8DWpLYMVrv27JyzAk6GRxX"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "test.json"

In [ ]:
experiment = Experiment(job="Data Preparation", workspace=1507, project=5598)

In [ ]:
experiment.add_dataset_version()

In [ ]:
experiment.start()

In [ ]:
experiment.add_dataset_version()

In [ ]:
experiment.complete()